# Get SOV and accuracy predictions from SNNS old cross-validation output

Modified version of sov-and-accuracy.ipynb to sync with output of wrapper.pl ran on 25 and 26 Aug 2021. Naming of files is based on seqIDs instead of SCOPe domain names.

This one is a control because Geoff said the Standard deviation for the previous runs looked a bit high.

I look at the standard deviation from old cross validation runs, namely the one from /homes/adrozdetskiy/Projects/JnetDatasets/Jnet_training_output/

In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import subprocess

In [2]:
# Write function calc() that outputs:
# 1. HMM Q3 accuracy (N matches/N total)
# 2. PSSM Q3 accuracy
# 3. HMM SOV (call calSOV binary from python) 
# 4. PSSM SOV
# 
# Given the following arguments
# 1. sequence
# 2. dssp
# 3. hmm prediction
# 4. pssm prediction

def calc(outfolder,scopeID,seq,dssp,hmm_pred,pssm_pred): 
    n=len(seq)
    assert n!=0
    assert all(_==n for _ in [len(dssp),len(hmm_pred),len(pssm_pred)])
    hmm_pred=hmm_pred.replace('-','C')
    pssm_pred=pssm_pred.replace('-','C')
    dssp=dssp.replace('-','C')
    # write two sov.in file
    hmm_sov_in=os.path.join(outfolder,'%s.hmm.sov.in' % scopeID)
    pssm_sov_in=os.path.join(outfolder,'%s.pssm.sov.in' % scopeID)
    with open(hmm_sov_in,'w+') as f_hmm, open(pssm_sov_in,'w+') as f_pssm:
        f_hmm.write("AA  OSEC PSEC\n")
        f_pssm.write("AA  OSEC PSEC\n")
        for (aa,osec,psec_hmm,psec_pssm) in zip(seq,dssp,hmm_pred,pssm_pred): # no commas allowed
            f_hmm.write("%s   %s    %s\n" % (aa,osec,psec_hmm))
            f_pssm.write("%s   %s    %s\n" % (aa,osec,psec_pssm))

    hmm_sov_out=os.path.join(outfolder,'%s.hmm.sov.out' % scopeID)
    pssm_sov_out=os.path.join(outfolder,'%s.pssm.sov.out' % scopeID)

    cmd_hmm="calSOV -f 1 %s -o %s" % (hmm_sov_in,hmm_sov_out) # must have calSOV installed and path configured
    cmd_pssm="calSOV -f 1 %s -o %s" % (pssm_sov_in,pssm_sov_out)
    proc_hmm=subprocess.run(args=cmd_hmm.split())
    proc_pssm=subprocess.run(args=cmd_pssm.split())

    assert proc_hmm.returncode==0
    assert proc_pssm.returncode==0
    assert os.path.exists(hmm_sov_out)
    assert os.path.exists(pssm_sov_out)

    with open(hmm_sov_out,'r') as f:
        lines=f.read().splitlines()
        ans=lines[1].split()
        hmm_sov,hmm_acc=ans[1],ans[10]
        
    assert os.path.exists(pssm_sov_out)
    
    with open(pssm_sov_out,'r') as f:
        lines=f.read().splitlines()
        ans=lines[1].split()
        pssm_sov,pssm_acc=ans[1],ans[10]
    
    assert(_!="nan" & _!="-nan" for _ in [hmm_sov,hmm_acc,pssm_sov,pssm_acc])    
    
#    del_cmd='rm %s %s %s %s' % (hmm_sov_in, hmm_sov_out, pssm_sov_in, pssm_sov_out)
#    proc_del=subprocess.run(args=del_cmd.split())
#    assert proc_del.returncode==0    
    
    return tuple([float(v) for v in (hmm_acc,pssm_acc,hmm_sov,pssm_sov)])

# SNNS

In [59]:
debug=False
irange=range(2,8)
expt_name='2014-07-17'
#root_folder="/cluster/gjb_lab/2472402/snns_cross_val_25_Aug/"
root_folder="/cluster/gjb_lab/2472402/snns_cross_val_2014-07-18"

In [60]:
for i in irange:

    # read SCOPe domain names into a list
    cross_val_folder=os.path.join(root_folder, 'cross-val%d' % i)
    assert os.path.exists(cross_val_folder)
    results=[]
    alignments=[]
    
    
    # get seqIDs from .jnet files sitting in valid directory
    # this works only after one has run scripts/resume_scoring.pl in each of the cross-val directories to create jnet files
    seqIDs=sorted(_[:-5] for _ in os.listdir(cross_val_folder) if _[-5:]=='.jnet')
    
    dssp_file="/cluster/gjb_lab/2472402/snns_cross_val_2014-07-18/cross-val%d-dssp.csv" % i
    dssp_dict=dict()
    assert os.path.exists(dssp_file)
    with open(dssp_file,'r') as f:
        lines=f.read().splitlines()
        lines=lines[1:]
        for line in lines:
            line=line.split(',')
            seqID=line[0]
            domain=line[1]
            dssp=line[2]
            dssp_dict[seqID]={'dssp':dssp,'dom':domain}
    
    for seqID in seqIDs:
        
        scopeID=dssp_dict[str(seqID)]['dom']
        dssp=dssp_dict[seqID]['dssp']
        
        jnet_path=os.path.join(cross_val_folder,seqID+'.jnet') # look for .jnet files in data dir
        with open(jnet_path,'r') as f:
            lines=f.read().splitlines()
            hmm_pred=lines[6].replace("JNETHMM:","").replace(",","")
            pssm_pred=lines[7].replace("JNETPSSM:","").replace(",","")
        
        fasta_file=os.path.join(cross_val_folder,'data',seqID+'.fasta')
        
        assert os.path.exists(fasta_file)
        with open(fasta_file, 'r') as f:
            seq=f.read().splitlines()[1]
        
        # check sanity of input to calc()
        if debug:
            print("scopeID:   %s" % seqID)
            print("DSSP:      %s" % dssp)
            print("HMM pred:  %s" % hmm_pred)
            print("PSSM pred: %s" % pssm_pred)
        
        sov_root='/cluster/gjb_lab/2472402/sov/snns/%s' % expt_name
        if not os.path.exists(sov_root):
            os.system("mkdir %s" % sov_root)
        sov_folder=os.path.join(sov_root,'cross-val%d' % i)
        if not os.path.exists(sov_folder):
            os.system("mkdir %s" % sov_folder)
        (hmm_acc,pssm_acc,hmm_sov,pssm_sov)=calc(sov_folder,seqID,seq,dssp,hmm_pred,pssm_pred)

        # check sanity of output of calc()
        if debug:
            print("output of calc: ")
            print("HMM acc:   %s" % hmm_acc)
            print("PSSM acc:  %s" % pssm_acc)
            print("HMM SOV:   %s" % hmm_sov)
            print("PSSM SOV:  %s" % pssm_sov)

        #store results
        results.append((seqID,hmm_acc,pssm_acc,hmm_sov,pssm_sov))
        alignments.append((seqID,seq,dssp,hmm_pred,pssm_pred))


    # output results to csv file
    out_folder="/cluster/gjb_lab/2472402/results/snns/%s" % expt_name
    if not os.path.exists(out_folder):
        os.system("mkdir %s" % out_folder)
    out_csv="cv%i_scores.csv" % i
    with open(os.path.join(out_folder,out_csv),'w+') as f:
        f.write("seqID,HMM_acc,PSSM_acc,HMM_sov,PSSM_sov\n")
        for (seqID,hmm_acc,pssm_acc,hmm_sov,pssm_sov) in results:
            line="%s,%.2f,%.2f,%.2f,%.2f\n" % (seqID,hmm_acc,pssm_acc,hmm_sov,pssm_sov)
            f.write(line)

    # write alignments to kerasnet file
    out_knet="cv%i.knet" % i
    with open(os.path.join(out_folder,out_knet),'w+') as f:
        for (seqID, seq,dssp,hmm_pred,pssm_pred) in alignments:
            f.write("seqID     : %s\n" % seqID)
            f.write("sequence  : %s\n" % seq)
            f.write("DSSP      : %s\n" % dssp)
            f.write("HMM_pred  : %s\n" % hmm_pred)
            f.write("PSSM_pred : %s\n" % pssm_pred)